In [38]:
import seaborn as sns
import pandas as pd

df = sns.load_dataset('titanic')

train_x = df[:-100][["pclass", "sex", "age", "embarked", "alone"]]
train_y = df[:-100][["survived"]]
test_x = df[-100:][["pclass", "sex", "age", "embarked", "alone"]]
test_y = df[-100:][["survived"]]

In [39]:
print(train_x)

     pclass     sex   age embarked  alone
0         3    male  22.0        S  False
1         1  female  38.0        C  False
2         3  female  26.0        S   True
3         1  female  35.0        S  False
4         3    male  35.0        S   True
..      ...     ...   ...      ...    ...
786       3  female  18.0        S   True
787       3    male   8.0        Q  False
788       3    male   1.0        S  False
789       1    male  46.0        C   True
790       3    male   NaN        Q   True

[791 rows x 5 columns]


In [40]:
test_y["survived"].value_counts() / len(test_y)

0    0.64
1    0.36
Name: survived, dtype: float64

Leg uit dat je matrix operaties kan doen hier

In [122]:
def gini_impurity(labels: pd.Series) -> float:
  # bereken het percentage hoevaak elke class voorkomt in data

  p = labels.value_counts() / len(labels) # bevat een lijst met een percentage voor elke class.

  # doe elk percentage in het kwadraat en tel dit allemaal bij elkaar op.
  pi = sum(p ** 2)

  return 1 - pi

In [125]:
print(gini_impurity(df["sex"]))

0.4564361661256535


In [137]:
def get_split_options(values: pd.Series):
  

  options = values.unique()
  return options

def get_max_information_gain_split(values, labels):

  options = get_split_options(values)

  best_gini = 1
  for val in options:
    if type(val) is str:
      mask = values == val
    else:
      mask = values < val
    left_labels = labels[mask]
    right_labels = labels[~mask]

    entropy_left = gini_impurity(left_labels)*(sum(mask)/len(labels)) 
    entropy_right = gini_impurity(right_labels)*(sum(~mask)/len(labels)) 

    print(entropy_left, entropy_right)

    candidate_gini = sum([entropy_left, entropy_right])

    if candidate_gini < best_gini:
      print(candidate_gini)
      best_gini = candidate_gini

  return best_gini

def get_best_split(targets, data):
  NotImplemented

In [139]:
print(get_max_information_gain_split(train_x["age"], train_y))

179 612
0.49024687119627974 0.4683401683113333
0.11094082167400009 0.3623567421068723
0.4732975637808724
464 327
0.4879607609988109 0.44646447642828413
0.28623741226731764 0.18456875321371544
0.47080616548103305
268 523
0.48652261082646464 0.4666854337200078
0.16483951921806894 0.30856698082877887
419 372
0.48482293903543494 0.45940282113539144
0.2568151851527778 0.21605290703206778
0 791
1 0.47439509910002053
0.0 0.47439509910002053
583 208
0.4861587165221588 0.422291050295858
0.3583192562988857 0.11104492852280462
0.46936418482169034
11 780
0.2975206611570247 0.4715680473372781
0.004137455464889092 0.4650102110279101
0.4691476664927992
283 508
0.4850104259011849 0.46725618451236905
0.17352458979776905 0.3000836178663508
61 730
0.47729105079279766 0.4654118971664477
0.03680752730513357 0.429520461354623
0.4663279886597566
26 765
0.4526627218934911 0.47012345679012346
0.01487892638335116 0.4546705998033432
599 192
0.48521603897425036 0.42165798611111116
0.3674392001840404 0.10234934681